# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Answer 1:  

Quote from Section "2 Materials and Methods" This research focus on targeting through telemarketing phone calls to sell long-term deposits. Within a campaign, the human agents execute phone calls to a list of clients to sell the deposit (outbound) or, if meanwhile the client calls the contact-center for any other reason, he is asked to subscribe the deposit (inbound). Thus, the result is a binary unsuccessful or successful contact.

The amount of unique Campaigns is 42

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('/Users/miguel/HaaS/Module 17_Cap/module_17_starter/data/bank-additional-full.csv', sep = ';')

In [8]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [9]:
num_campaigns = df['campaign'].nunique()
num_campaigns

42

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Answer 3:
No missing values, which is a good thing, but for the 'unknown' values within each categorical column, I may try to impute later during the analysis.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Answer 4:

The business objective is to develop a model that can predict if a client will subscribe to a term deposit based on various features such as: demographics, previous interactions with the bank, social and economic attributes. This model will assist the bank by narrowing its marketing efforts by identifying potential clients who are more likely to subscribe to a term deposit. Resulting in an improved marketing campaign rate. 


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [16]:
#Here I need to import the required packages to perform encoding and scaling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#Here I select features for columns 1 - 7
bank_info_features = df.iloc[:, :7]

# My target variable is column 'Y'
y = df['y']

# Creating datasets based on categorical and numerical features
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan']
numerical_features = ['age']




In [17]:
# Here I create preprocessing pipeline to encode and scale the numerical values
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(drop='first'), categorical_features)
])
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(), ['age']),
                                ('cat', OneHotEncoder(drop='first'),
                                 ['job', 'marital', 'education', 'default',
                                  'housing', 'loan'])])

In [18]:
# This is where I fit the preprocessing pipeline
X_processed = preprocessor.fit_transform(bank_info_features)
X_processed

<41188x28 sparse matrix of type '<class 'numpy.float64'>'
	with 183946 stored elements in Compressed Sparse Row format>

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [20]:
from sklearn.model_selection import train_test_split

# Here I split my datasets into training and test sets where 80% is trained and tested on the remaining 20%, you will notice that stratify is included 
# as the initial split create an issue where there was only 'no' within the y_pred dataset
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)


# Below are the sizes of each training and test dataset
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)



Shape of X_train: (32950, 28)
Shape of X_test: (8238, 28)
Shape of y_train: (32950,)
Shape of y_test: (8238,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

### Answer 7: 

Basically, the classifier that we should aim to beat is predicting 'no' for every instance in the target variable ('y). 
The assumption is based on the liklihood that most instances will be 'no', that clients will not subscribe to a term deposit. So, our goal is for the training classifier to achieve better accuracy along with performance metrics.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [24]:
from sklearn.linear_model import LogisticRegression

# Here is where I set the logistic regression model
log_reg = LogisticRegression(random_state=42, max_iter=1000)

# I then fit the logistic regression model with the training data
log_reg.fit(X_train, y_train)

# I then predict on the test data
y_pred = log_reg.predict(X_test)


### Problem 9: Score the Model

What is the accuracy of your model?

In [26]:
from sklearn.metrics import accuracy_score

# Here I ran accuracy against the logistic regression
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8865015780529255


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [28]:
#In order to compare the performance of LR, KNN, Decision Tree and SVM, I need to import the appropriate packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time

# I then intialize each model
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42)
}

# Below are lists to store results from the For Loop
train_times = []
train_accuracies = []
test_accuracies = []

# Below is where I fit and score each model
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_accuracy = accuracy_score(y_train, train_pred)
    test_accuracy = accuracy_score(y_test, test_pred)
    
    train_times.append(train_time)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

# Below I create a DataFrame to present my results within the table formate above. 

results_df = pd.DataFrame({
    "Model": list(models.keys()),
    "Train Time (s)": train_times,
    "Train Accuracy": train_accuracies,
    "Test Accuracy": test_accuracies
})

results_df


,Model,Train Time (s),Train Accuracy,Test Accuracy
0,Logistic Regression,0.090948,0.887557,0.886502
1,K-Nearest Neighbors,0.014494,0.889742,0.875212
2,Decision Tree,0.141324,0.916601,0.861131
3,Support Vector Machine,23.537822,0.888376,0.886623


While each model performed farily well, Logistic regression can be seen as the most consistent. KNN came in second in terms of accuracy compared, while the decision tree model had the highest training accuracy but dropped in test accuracy, which might mean overfitting. The support vector machine (SVM) model had the longest training time and achieved similar accuracy to logistic regression.

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [31]:
#The following is a gridsearch performed on KNN

from sklearn.model_selection import GridSearchCV

# Below are my chose hyperparameters and their possible values
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# I want my results printed out with the best hyperparameters
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best hyperparameters
best_model = grid_search.best_estimator_
best_model

Best Parameters: {'n_neighbors': 9, 'weights': 'uniform'}


KNeighborsClassifier(n_neighbors=9)

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score

# This was my attempt to use other metrics, all of my y_pred data consisted of only 'no' values. Even with Stratify and Cross Valdiation, I kept running into the same issue
precision = precision_score(y_test, y_pred, pos_label='yes')
recall = recall_score(y_test, y_pred, pos_label='yes')
f1 = f1_score(y_test, y_pred, pos_label='yes')


print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


Precision: 0.0
Recall: 0.0
F1-Score: 0.0


/Users/miguel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
import numpy as np
# Unique values in y_pred
unique_values_pred = np.unique(y_pred)
print("Unique values in y_pred:", unique_values_pred)

# Unique values in y_test
unique_values_test = np.unique(y_test)
print("Unique values in y_test:", unique_values_test)


Unique values in y_pred: ['no']
Unique values in y_test: ['no' 'yes']


In [ ]:
# I created a set of parameters for the three models below
param_grid = {
    'KNN': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']},
    'Decision Trees': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
    'SVM': {'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf']}
}

# Setting the models
models = {
    'KNN': KNeighborsClassifier(),
    'Decision Trees': DecisionTreeClassifier(),
    'SVM': SVC()
}

# Here I performed GridSearchCV across for each model
for model_name, model in models.items():
    print(f"Tuning hyperparameters for {model_name}...")
    clf = GridSearchCV(model, param_grid[model_name], cv=5, scoring='accuracy')
    clf.fit(X_train, y_train)
    print(f"Best parameters: {clf.best_params_}")
    print(f"Best cross-validation score: {clf.best_score_}")
    print()

##### Questions